In [31]:
import pandas as pd
import numpy as np

In [12]:
n = 100000
n_test=n
read_all = False
if(read_all):
    n = 10000000
    n_test=18790469
label_col='is_attributed'


In [13]:
def read_train_data():
    file='Resources/Data/train.csv'
    train_data= pd.read_csv(file,nrows=n)
    return train_data.loc[:, train_data.columns != label_col], train_data[[label_col]]

In [14]:
def read_test_data():
    file='Resources/Data/test.csv'
    test_data= pd.read_csv(file,nrows=n_test)
    return test_data

In [15]:
import datetime
def datetime_to_float(a):
    if(isinstance(a, str)):
        d=datetime.datetime.strptime(a, "%Y-%m-%d %H:%M:%S")
        epoch = datetime.datetime.utcfromtimestamp(0)
        total_seconds =  (d - epoch).total_seconds()
        return total_seconds
    return 0.0
date_transformer = lambda x: datetime_to_float(x)

In [19]:
def preprocess(data):
    data['click_time']=data['click_time'].map(date_transformer)
    if 'attributed_time' in data.columns:
        data['attributed_time']=data['attributed_time'].map(date_transformer)
    return data

In [20]:
X_train_raw, y_train_raw=read_train_data();
X_train_raw.shape

(100000, 7)

In [21]:
X_train=preprocess(X_train_raw)
X_train.head(5)

,ip,app,device,os,channel,click_time,attributed_time
0,83230,3,1,13,379,1.509979e+09,0.0
1,17357,3,1,19,379,1.509979e+09,0.0
2,35810,3,1,13,379,1.509979e+09,0.0
3,45745,14,1,13,478,1.509979e+09,0.0
4,161007,3,1,13,379,1.509979e+09,0.0


In [22]:
y_train=y_train_raw
input_dim=X_train.shape[1]

In [23]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Activation
model = Sequential()
model.add(Dense(100, input_dim=input_dim, activation='relu'))
model.add(Dense(70, input_dim=input_dim, activation='relu'))
model.add(Dense(35, input_dim=input_dim, activation='relu'))
model.add(Dense(17, input_dim=input_dim, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics = ['accuracy'])

In [24]:
model.fit(X_train,y_train,epochs=2)

Epoch 1/2
100000/100000 [==============================] - 9s 88us/step - loss: 0.0272 - acc: 0.9983
Epoch 2/2
100000/100000 [==============================] - 8s 85us/step - loss: 0.0272 - acc: 0.9983


In [ ]:
score = model.evaluate(X_train, y_train, verbose=0)

In [141]:
X_test_raw=read_test_data()

In [ ]:
def combine_data_series(a,b):
    return pd.concat([a, b],axis=1)
def array_to_series(a):
    return pd.Series(a.reshape(a.shape[0]))
def combine_array_to_series(ser,arr):
    arr_ser=array_to_series(arr)
    return combine_data_series(ser,arr_ser)
def series_to_np_array(a):
    return a.values.reshape((-1,1))

In [131]:
X_test=preprocess(X_test_raw)
predictions= model.predict(X_test)

In [139]:
def write_result():
    submission_data= combine_array_to_series(X_test['click_id'],predictions)
    submission_data.columns = ['click_id', 'is_attributed']
    submission_data.to_csv('submission.csv',index=False)

In [140]:
write_result()